In [1]:
from syft import Worker, MessageStatus
from syft.core.node.new.project import Project
import syft as sy

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/PySyft


In [2]:
worker = Worker.named("test-domain-1", processes=1, reset=False)

SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/7bca415d13ed1ec841f0d0aede098dbb.sqlite

> Starting Worker: test-domain-1 - 7bca415d13ed1ec841f0d0aede098dbb - NodeType.DOMAIN - [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.metadata_service.MetadataService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>, <class 'syft.core.node.new.user_code_service.UserCodeService'>, <class 'syft.core.node.new.request_service.RequestService'>, <class 'syft.core.node.new.data_subject_service.DataSubjectService'>, <class 'syft.core.node.new.network_service.NetworkService'>, <class 'syft.core.node.new.policy_service.PolicyService'>, <class 'syft.core.node.new.message_service.MessageService'>, <class 'syft.core.node.new.project_service.ProjectService'>, <class 'syft.core.node.new.data_subject_member_service.Data

In [3]:
domain_client = worker.guest_client.login(email="info@openmined.org", password="changethis")

# Get messages

In [4]:
messages = domain_client.api.services.messages.get_all_for_status(MessageStatus.UNDELIVERED)

In [5]:
messages

,type,id,subject,status,created_at,linked_obj
0,syft.core.node.new.messages.Message,1b70c191e067449b8d892ca93f64171e,Approval Request,MessageStatus.UNDELIVERED,2023-05-02 10:42:22,<<class 'syft.core.node.new.request.Request'>:...
1,syft.core.node.new.messages.Message,466a8754a550447197431d948625833f,Project Approval,MessageStatus.UNDELIVERED,2023-05-02 10:42:22,<<class 'syft.core.node.new.project.Project'>:...


In [6]:
project_message = [x for x in messages if issubclass(x.linked_obj.object_type, Project)][0]

In [7]:
request = project_message.link.requests[0]
func = request.changes[0].link
op = func.output_policy_type

In [8]:
func

```python
class UserCode:
  id: str = fe482f7823444b938a5c3f3394f700b2
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  user_verify_key: str = f924434267cec265a8879ce91511ad3bf658c81ab36e78e62fef63844e57e4de
  raw_code: str = "@sy.syft_function(input_policy=sy.ExactMatch(df=ds.assets[0]),
                  output_policy=sy.SingleExecutionExactOutput())
def snow_and_temperature(df):
    import pandas as pd
    import numpy as np
    weather_2012 = df
    weather_description = weather_2012['Weather']
    is_snowing = weather_description.str.contains('Snow')
    
    median_temperature = weather_2012['Temp (C)'].resample('M').apply(np.median)
    snow_frequency = is_snowing.astype(float).resample('M').apply(np.mean)
    
    temperature = weather_2012['Temp (C)'].resample('M').apply(np.median)
    is_snowing = weather_2012['Weather'].str.contains('Snow')
    snowiness = is_snowing.astype(float).resample('M').apply(np.mean)

    # Name the columns
    temperature.name = "Temperature"
    snowiness.name = "Snowiness"
    stats = pd.concat([temperature, snowiness], axis=1)
    stats.columns = ["temperature", "snowiness"]
    
    return is_snowing.astype(float), median_temperature, snow_frequency, stats
"
  input_policy_type: str = <class 'syft.core.node.new.policy.ExactMatch'>
  input_policy_init_kwargs: str = {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): {'df': <UID: 1f625795d2a74f32b2f07b3e99eaa480>}}
  input_policy_state: str = b''
  output_policy_type: str = <class 'syft.core.node.new.policy.OutputPolicyExecuteOnce'>
  output_policy_init_kwargs: str = {}
  output_policy_state: str = b''
  parsed_code: str = "

def user_func_snow_and_temperature_f924434267cec265a8879ce91511ad3bf658c81ab36e78e62fef63844e57e4de_413ce3ef2dd8af7721c0b58ddbaa8463e90391cecfd0894282f5ae9c4d860719(df):

    def snow_and_temperature(df):
        import pandas as pd
        import numpy as np
        weather_2012 = df
        weather_description = weather_2012['Weather']
        is_snowing = weather_description.str.contains('Snow')
        median_temperature = weather_2012['Temp (C)'].resample('M').apply(np.median)
        snow_frequency = is_snowing.astype(float).resample('M').apply(np.mean)
        temperature = weather_2012['Temp (C)'].resample('M').apply(np.median)
        is_snowing = weather_2012['Weather'].str.contains('Snow')
        snowiness = is_snowing.astype(float).resample('M').apply(np.mean)
        temperature.name = 'Temperature'
        snowiness.name = 'Snowiness'
        stats = pd.concat([temperature, snowiness], axis=1)
        stats.columns = ['temperature', 'snowiness']
        return (is_snowing.astype(float), median_temperature, snow_frequency, stats)
    result = snow_and_temperature(df=df)
    return result
"
  service_func_name: str = "snow_and_temperature"
  unique_func_name: str = "user_func_snow_and_temperature_f924434267cec265a8879ce91511ad3bf658c81ab36e78e62fef63844e57e4de_413ce3ef2dd8af7721c0b58ddbaa8463e90391cecfd0894282f5ae9c4d860719"
  user_unique_func_name: str = "user_func_snow_and_temperature_f924434267cec265a8879ce91511ad3bf658c81ab36e78e62fef63844e57e4de"
  code_hash: str = "413ce3ef2dd8af7721c0b58ddbaa8463e90391cecfd0894282f5ae9c4d860719"
  signature: str = (df)
  status: str = {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): <UserCodeStatus.SUBMITTED: 'submitted'>}
  input_kwargs: str = ['df']
  enclave_metadata: str = None

```

In [9]:
get_col_user_function = func.unsafe_function

In [10]:
real_data = domain_client.datasets[0].assets[0].data

In [11]:
real_data

,Temp (C),Dew Point Temp (C),Rel Hum (%),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Weather
Date/Time,,,,,,,
2012-01-01 00:00:00,-1.8,-3.9,86,4,8.0,101.24,Fog
2012-01-01 01:00:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2012-01-01 02:00:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog"
2012-01-01 03:00:00,-1.5,-3.2,88,6,4.0,101.27,"Freezing Drizzle,Fog"
2012-01-01 04:00:00,-1.5,-3.3,88,7,4.8,101.23,Fog
...,...,...,...,...,...,...,...
2012-12-31 19:00:00,0.1,-2.7,81,30,9.7,100.13,Snow
2012-12-31 20:00:00,0.2,-2.4,83,24,9.7,100.03,Snow
2012-12-31 21:00:00,-0.5,-1.5,93,28,4.8,99.95,Snow


In [12]:
real_result = get_col_user_function(df=real_data)

In [14]:
result = request.accept_by_depositing_result(real_result)
result
assert isinstance(result, sy.SyftSuccess)